<a href="https://colab.research.google.com/github/cluciani-angel/documentation/blob/main/Casino_Data_ETL_Script_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import uuid
import os
import logging
from io import StringIO

# --- 1. Configuration & Logging Setup ---

# Set up basic logging to capture informational messages and errors.
# This will create a log file in the same directory as the script.
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("etl_process.log"),
        logging.StreamHandler() # Also print logs to the console
    ]
)

# --- 2. Helper Function to Safely Convert IDs ---

def safe_hex_to_uuid(hex_string):
    """
    Safely converts a hex string to a UUID object.

    This function is the core solution to the "Got bytestring" error.
    It validates the input string before trying to convert it.

    Args:
        hex_string (str): The string to convert, expected to be 32 hex characters.

    Returns:
        uuid.UUID: The UUID object if conversion is successful.
        pd.NA: Returns pandas' Not Available marker if the string is invalid.
    """
    if not isinstance(hex_string, str) or len(hex_string) != 32:
        # The bytestring must be 16 bytes, which means the hex string must be 32 characters.
        # This check catches the length error before it happens.
        return pd.NA
    try:
        # Convert the 32-character hex string into a 16-byte object, then into a UUID.
        return uuid.UUID(bytes=bytes.fromhex(hex_string))
    except (ValueError, TypeError):
        # Catches other errors, like non-hex characters in the string.
        return pd.NA

# --- 3. The ETL Pipeline Functions ---

def extract_data(file_path):
    """
    Extracts data from a given CSV file path.
    """
    logging.info(f"Starting data extraction from: {file_path}")
    if not os.path.exists(file_path):
        logging.error(f"File not found at {file_path}. Creating a dummy dataframe for demonstration.")
        # Create a dummy CSV in memory to demonstrate the script's functionality
        dummy_csv_data = """id_hex,player_name,transaction_amount,timestamp
d3d6e0a01b3a4c9f8f1e7d6a5b4c3d2e,Player1,100,2025-08-10T10:00:00Z
636f727275707465,Player2,250,2025-_08-10T11:30:00Z
e1c2a3b45d6e7f8a9b0c1d2e3f4a5b6c,Player3,50,2025-08-10T12:15:00Z
invalid-hex-string,Player4,120,2025-08-10T13:00:00Z
d3d6e0a01b3a4c9f8f1e7d6a5b4c3d2f,Player5,300,2025-08-10T14:00:00Z
"""
        return pd.read_csv(StringIO(dummy_csv_data), dtype=str)

    try:
        # Reading all columns as strings initially prevents pandas from making wrong type guesses.
        df = pd.read_csv(file_path, dtype=str, sep=';', engine='python')
        logging.info("Data extraction successful.")
        return df
    except Exception as e:
        logging.error(f"Failed to read data from {file_path}. Error: {e}")
        return None

def transform_data(df, id_column_name):
    """
    Cleans, validates, and transforms the raw dataframe.
    """
    if df is None:
        return None

    logging.info("Starting data transformation...")

    df_transformed = df.copy()

    # --- NEW DEBUGGING STEP ---
    # Log the columns that were actually found in the file. This helps diagnose mismatches.
    logging.info(f"Columns found in the CSV file: {df_transformed.columns.to_list()}")
    # --- END NEW DEBUGGING STEP ---

    # Check if the required ID column exists
    if id_column_name not in df_transformed.columns:
        logging.error(f"The specified ID column '{id_column_name}' was not found in the data. Please check the column list above and update the 'ID_COLUMN_TO_VALIDATE' variable in the script.")
        return None

    # Apply the safe UUID conversion function
    logging.info(f"Converting '{id_column_name}' to UUID. Invalid entries will be flagged.")
    df_transformed['uuid'] = df_transformed[id_column_name].apply(safe_hex_to_uuid)

    invalid_rows = df_transformed[df_transformed['uuid'].isna()]
    if not invalid_rows.empty:
        logging.warning(f"Found {len(invalid_rows)} rows with invalid IDs. See details below.")
        for index, row in invalid_rows.iterrows():
            logging.warning(f"  - Row Index {index}: Invalid ID value = '{row[id_column_name]}'")

    logging.info("Standardizing column names and types.")

    logging.info("Data transformation complete.")
    return df_transformed

def load_data(df, output_path):
    """
    Saves the transformed dataframe to a new CSV file.
    """
    if df is None:
        logging.warning("No data to load; skipping file creation.")
        return

    logging.info(f"Loading transformed data to: {output_path}")
    try:
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        df.to_csv(output_path, index=False)
        logging.info("Data loading successful.")
    except Exception as e:
        logging.error(f"Failed to save data to {output_path}. Error: {e}")

# --- 4. Main Execution Block ---

if __name__ == "__main__":
    logging.info("====== Starting Casino Data ETL Process ======")

    # --- Configuration: SET YOUR FILE PATHS AND COLUMN NAME HERE ---

    INPUT_CSV_PATH = '/content/drive/MyDrive/Reportes Auditoria/SIELCON/08-2025/Tickets08-2025.csv'

    # IMPORTANT: Change this to your actual column name.
    # Look at the log output from the line "Columns found in the CSV file:" to find the correct name.
    ID_COLUMN_TO_VALIDATE = 'id_hex'

    OUTPUT_CSV_PATH = 'processed/tickets_cleaned.csv'

    # --- Run the Pipeline ---
    raw_dataframe = extract_data(INPUT_CSV_PATH)
    transformed_dataframe = transform_data(raw_dataframe, ID_COLUMN_TO_VALIDATE)
    load_data(transformed_dataframe, OUTPUT_CSV_PATH)

    logging.info("====== ETL Process Finished ======")